<a href="https://colab.research.google.com/github/sRajdeep617/Analyze-URLs/blob/main/Analyze_URLs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Analyze and Visualize URLs with Network Graph**

In [14]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [1]:
!pip install yarl

     |████████████████████████████████| 271 kB 25.9 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 


In [8]:
from yarl import URL

In [10]:
def site(url):
  # getting scheme
  print('\nScheme:', url.scheme)

  # getting host
  print('\nHost:', url.host)

  # getting path
  print('\nPath:', url.path)

  # getting query
  print('\nQuery:', url.query)

  # getting fragment
  print('\nFragment:', url.fragment)

Starting from a simple site

In [12]:
url = URL("https://github.com/search?q=data+science")
url1 = URL("https://github.com/khuyentran1401/Data-science#contents")

site(url)
print('\n')
site(url1)


Scheme: https

Host: github.com

Path: /search

Query: <MultiDictProxy('q': 'data science')>

Fragment: 



Scheme: https

Host: github.com

Path: /khuyentran1401/Data-science

Query: <MultiDictProxy()>

Fragment: contents


In [16]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Projects/Analyze URLs /URL Classification.csv', names = ['url', 'Type'], index_col = 0)
sample = data.sample(1000, random_state = 1)
sample.head(10)

,url,Type
1308349,http://yorkrite.com/ne/gcram/,Society
276825,http://www.alliedartistsofamerica.org/,Arts
1315267,http://www.msstate.edu/org/farmhouse/index.html,Society
392387,http://www.sunnytec.com.tw/,Business
1267730,http://www.kabissa.org,Society
1392788,http://www.newadvent.org/cathen/12134b.htm,Society
672881,http://www.gamespot.com/ps2/driving/arcticthun...,Games
1526125,http://www.studsquad.net/,Sports
1009901,http://www.spacedog.biz,Reference
330964,http://www.meditrans-japan.com/,Business


Extract some elements from the 'url' column of dataset

In [17]:
sample['url'] = sample['url'].apply(lambda url: URL(url))

processed = sample.assign(
    host = sample.url.apply(lambda url: url.host),
    path = sample.url.apply(lambda url: url.path),
    name = sample.url.apply(lambda url: url.name),
    scheme = sample.url.apply(lambda url: url.scheme),
    query = sample.url.apply(lambda url: url.query_string),
)

processed

,url,Type,host,path,name,scheme,query
1308349,http://yorkrite.com/ne/gcram/,Society,yorkrite.com,/ne/gcram/,,http,
276825,http://www.alliedartistsofamerica.org/,Arts,www.alliedartistsofamerica.org,/,,http,
1315267,http://www.msstate.edu/org/farmhouse/index.html,Society,www.msstate.edu,/org/farmhouse/index.html,index.html,http,
392387,http://www.sunnytec.com.tw/,Business,www.sunnytec.com.tw,/,,http,
1267730,http://www.kabissa.org,Society,www.kabissa.org,/,,http,
...,...,...,...,...,...,...,...
1195234,http://www.fiberglassspecialtiesinc.com,Shopping,www.fiberglassspecialtiesinc.com,/,,http,
1492482,http://www.clemson.edu/fencers/,Sports,www.clemson.edu,/fencers/,,http,
1305913,http://www.mofreemason.com/gower397/,Society,www.mofreemason.com,/gower397/,,http,
68035,http://www.waterlooschool.co.uk/,Arts,www.waterlooschool.co.uk,/,,http,


**Visualize URLs’ Features with Network Graph**

Finding the top 5 most popular hosts for each article topic

In [20]:
# group by the columns Type and host and get the count
group = processed.groupby(['Type', 'host']).agg(count = ('url', 'count'))

# get the top 5 most popular hosts for each type
sorted_group = group.sort_values(by="count", ascending=False).reset_index()
largest = sorted_group.groupby('Type').head(5).sort_values(by = 'Type')

largest.tail(10)

,Type,host,count
8,Society,members.tripod.com,3
9,Society,www.angelfire.com,3
10,Society,www.geocities.com,3
0,Society,www.newadvent.org,15
14,Society,www.fourdir.com,2
304,Sports,baseball_gal.tripod.com,1
308,Sports,www.historicalkits.co.uk,1
305,Sports,www.geocities.com,1
306,Sports,www.grandstrandprogolftour.com,1
307,Sports,www.highonbikes.com,1


We will use PyGraphistry to visualize the relationship between 'Type' and 'host'.

In [21]:
!pip install graphistry

     |████████████████████████████████| 133 kB 17.8 MB/s 


In [23]:
import graphistry

graphistry.register(api = 3, username = 'Rajdeep617', password = 'P@ssword94313')

In [26]:
edges = largest[['Type', 'host']]
edges

,Type,host
622,Adult,www.caribbeancom.com
623,Adult,www.danskshowservice.dk
624,Adult,www.dvdxgay.com
27,Adult,alpha-cow.com
621,Adult,www.atlanticcompanions.com
...,...,...
304,Sports,baseball_gal.tripod.com
308,Sports,www.historicalkits.co.uk
305,Sports,www.geocities.com
306,Sports,www.grandstrandprogolftour.com


Creating Nodes


In [30]:
def create_node_df(df: pd.DataFrame, col_name: str):
  nodes = (
      df[[col_name]]
      .assign(type = col_name)
      .rename(columns = {col_name: 'node'})
      .drop_duplicates()
  )

  return nodes

type_nodes = create_node_df(largest, 'Type')
url_nodes = create_node_df(largest, 'host')
nodes = pd.concat([type_nodes, url_nodes])

nodes

,node,type
622,Adult,Type
1,Arts,Type
483,Business,Type
19,Computers,Type
709,Games,Type
...,...,...
14,www.fourdir.com,host
304,baseball_gal.tripod.com,host
308,www.historicalkits.co.uk,host
306,www.grandstrandprogolftour.com,host


Create a graph network

In [32]:
g = (
    graphistry
     .edges(edges, 'Type', 'host')
     .nodes(nodes, 'node')
)

g.plot()

Customizing the graph

In [33]:
!pip install faker

     |████████████████████████████████| 1.5 MB 25.7 MB/s 


In [34]:
from faker import Faker

types = list(edges.Type.unique())
fake = Faker()
colors = [fake.color() for __ in range(len(types))]
node_color_mapping = dict(zip(types, colors))
node_color_mapping

{'Adult': '#f9acaf',
 'Arts': '#e557a3',
 'Business': '#ef1096',
 'Computers': '#7082cc',
 'Games': '#94a825',
 'Health': '#43dd3e',
 'Home': '#ef9432',
 'Kids': '#a01828',
 'News': '#fc1ef1',
 'Recreation': '#e04a9a',
 'Reference': '#efe417',
 'Science': '#003c5e',
 'Shopping': '#5fe296',
 'Society': '#db6c60',
 'Sports': '#747af7'}

In [35]:
node_icon_mapping = {'host': 'link', 'Type': 'newspaper-o'}

In [36]:
g = (
    graphistry.edges(edges, 'Type', 'host')
    .nodes(nodes, 'node')
    .encode_point_color(
        'node', categorical_mapping = node_color_mapping, default_mapping = 'silver'
    )
    .encode_point_icon(
        'type', categorical_mapping = node_icon_mapping
    )
)

g.plot()

In the code above,

*   encode_point_color("node", …) tells the graph to encode a point’s color based on its value in the column node .

*   encode_point_color(…, categorical_mapping=node_color_mapping, default_mapping="silver") tells the graph which color to use for a particular type of node. If the color of a node is not specified, that node will have a silver color.







From the graph, we can see that www.geocities.com is the host that has the largest number of connections with other topics.